In [1]:
class treeNode():
    def _init_(self,feat,val,right,left):
        deatureToSplitOn = feat
        valueOfSplit = val
        rightBranch = right
        leftBranch = left

In [2]:
from numpy import *
def loadDataSet(filename):
    numPeat = len(open(filename).readline().split('\t'))
    dataMat = []
    fr = open(filename)
    for line in fr.readlines():
        lineArr = []
        curLine = line.strip().split('\t')
        for i in range(numPeat):
            lineArr.append(float(curLine[i]))
        dataMat.append(lineArr)
    return dataMat
    

In [3]:
def binSplitDataSet(dataSet1,feature,value): 
    dataSet = mat(dataSet1)
    mat0 = dataSet[nonzero(dataSet[:, feature] > value)[0],:]  
    mat1 = dataSet[nonzero(dataSet[:, feature] <= value)[0],:]  
    return mat0, mat1 


def regLeaf(dataSet):
    return mean(dataSet[:,-1])

def regErr(dataSet):
    return var(dataSet[:,-1])*shape(dataSet)[0]



In [4]:
def createTree(dataSet,leafType=regLeaf,errType=regErr,ops=(1,4)):
    feat,val = chooseBestSplit(dataSet,leafType,errType,ops)
    if feat == None:return val
    retTree = {}
    retTree['spInd'] = feat
    retTree['spVal'] = val
    lSet,rSet = binSplitDataSet(dataSet,feat,val)
    retTree['left'] = createTree(lSet,leafType,errType,ops)
    retTree['right'] = createTree(rSet,leafType,errType,ops)
    return retTree


def chooseBestSplit(dataSet1,leafType=regLeaf,errType=regErr,ops=(1,4)):
    dataSet = mat(dataSet1)
    tolS = ops[0];tolN = ops[1]#从信息中提取出来这里分别设定是1,4
    if len(set(dataSet[:,-1].T.tolist()[0])) ==1:#如果所有值相等退出
        return None,leafType(dataSet)
    m,n = shape(dataSet)
    S=errType(dataSet)#求总方差
    bestS = inf;bestIndex = 0;bestValue=0#初始化
    for featIndex in range(n-1):#对每一个特征
        for splitVal in set((dataSet[:,featIndex].T.A.tolist())[0]):
            mat0,mat1 = binSplitDataSet(dataSet,featIndex,splitVal)
            if (shape(mat0)[0]<tolN) or (shape(mat1)[0]<tolN) :continue#行数小于设定的值
            newS = errType(mat0) + errType(mat1)#得到新的误差值判断
            if newS < bestS:
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
    if (S-bestS)<tolS:#如果误差减少不大就退出
            return None,leafType(dataSet)
    mat0,mat1 = binSplitDataSet(dataSet,bestIndex,bestValue)#如果切分的数据集很小退出
    if (shape(mat0)[0] < tolN) or(shape(mat1)[0] < tolN):
            return None,leafType(dataSet)
    return bestIndex,bestValue

def isTree(obj):#对于对象进行判断
    return (type(obj).__name__=='dict')#注意name前后都是两个小短杠

def getMean(tree):
    if isTree(tree['right']) : tree['right'] = getMean(tree['right'])
    if isTree(tree['left']) : tree['left'] = getMean(tree['left'])
    return (tree['left']+tree['right'])/2.0

def prune(tree,testData):
    if shape(testData)[0] == 0:return getMean(tree)#如果没有测试数据就对于树进行坍塌处理
    if (isTree(tree['left']) or isTree(tree['right'])):
        lSet,rSet = binSplitDataSet(testData,tree['spInd'],tree['spVal'])
    if isTree(tree['left']): tree['left'] = prune(tree['left'],lSet)#将分出来的数据作为测试数据
    if isTree(tree['right']): tree['right'] = prune(tree['right'],rSet)
    if not isTree(tree['left']) and not isTree(tree['right']):
        lSet,rSet = binSplitDataSet(testData,tree['spInd'],tree['spVal'])
        #原来的总的误差
        errorNoMerge = sum(power(lSet[:,-1] - tree['left'],2)) + sum(power(rSet[:,-1] - tree['right'],2))
        treeMean = (tree['left']+tree['right'])/2.0
        #两个叶结点合并后的误差
        errorMerge = sum(power(testData[:,-1] - treeMean,2))
        if errorMerge < errorNoMerge: 
            print("merging")
            return treeMean
        else: return tree
    else: return tree
    
def linearSolve(dataSet):
    m,n = shape(dataSet)
    X = mat(ones((m,n)));Y = mat(ones((m,1)))#将X和Y中的数据格式化
    X[:,1:n] = dataSet[:,0:n-1]; Y = dataSet[:,-1]
    xTx = X.T*X
    if linalg.det(xTx) == 0.0:
        raise NameError('This matrix is singular, cannot do inverse,\n\
        try increasing the second value of ops')
    ws = xTx.I * (X.T * Y)
    return ws,X,Y

def modelLeaf(dataSet):#当数据不再需要切分的时候负责生成叶结点
    ws,X,Y = linearSolve(dataSet)
    return ws

def modelErr(dataSet):
    ws,X,Y = linearSolve(dataSet)
    yHat = X * ws
    return sum(power(Y - yHat,2))


#如果预测回归树使用regTreeEval，如果预测模型树使用modelTreeEval
def regTreeEval(model, inDat):#为了与下面保持一致
    return float(model)#返回浮点数

def modelTreeEval(model, inDat):
    n = shape(inDat)[1]
    X = mat(ones((1,n+1)))
    X[:,1:n+1]=inDat
    return float(X*model)

def treeForeCast(tree, inData, modelEval=regTreeEval):
    if not isTree(tree): return modelEval(tree, inData)#如果输入的是单个点或者行向量就返回一个浮点数
    if inData[tree['spInd']] > tree['spVal']:
        if isTree(tree['left']): return treeForeCast(tree['left'], inData, modelEval)#递归
        else: return modelEval(tree['left'], inData)
    else:
        if isTree(tree['right']): return treeForeCast(tree['right'], inData, modelEval)
        else: return modelEval(tree['right'], inData)
        
def createForeCast(tree, testData, modelEval=regTreeEval):
    m=len(testData)
    yHat = mat(zeros((m,1)))
    for i in range(m):
        yHat[i,0] = treeForeCast(tree, mat(testData[i]), modelEval)
    return yHat

In [5]:
from tkinter import *#这里在3.3后需要这样小写
from tkinter import ttk
root = Tk()
myLabel = Label(root,text="hello world")#一个小部件
myLabel.grid()#将myLabel的位置告诉布局管理器，这个会安排在二维表格中，没做设定将会被设定在（0,0）的位置
root.mainloop()

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

def reDraw(tolS,tolN):
    reDraw.f.clf()        # 清除上一个图避免重叠
    reDraw.a = reDraw.f.add_subplot(111)
    if chkBtnVar.get():#检查复选框，如果选上就使用线性模型回归树如果没选上使用标准回归方法
        if tolN < 2: tolN = 2
        myTree=regTrees.createTree(reDraw.rawDat, regTrees.modelLeaf,regTrees.modelErr, (tolS,tolN))
        yHat = regTrees.createForeCast(myTree, reDraw.testDat,regTrees.modelTreeEval)
    else:
        myTree=regTrees.createTree(reDraw.rawDat, ops=(tolS,tolN))
        yHat = regTrees.createForeCast(myTree, reDraw.testDat)
    reDraw.a.scatter(reDraw.rawDat[:,0], reDraw.rawDat[:,1], s=5) #真实值使用scatter绘制
    reDraw.a.plot(reDraw.testDat, yHat, linewidth=2.0) #预测值使用plot绘制
    reDraw.canvas.show()

def getInputs():
    try: tolN = int(tolNentry.get())#期望得到的是整数
    except: 
        tolN = 10 
        print("enter Integer for tolN")
        tolNentry.delete(0, END)#清除错误的文本替换默认的值
        tolNentry.insert(0,'10')
    try: tolS = float(tolSentry.get())#期望得到的是浮点数
    except: 
        tolS = 1.0 
        print("enter Float for tolS")
        tolSentry.delete(0, END)
        tolSentry.insert(0,'1.0')
    return tolN,tolS

def drawNewTree():
    #取得输入框的值
    tolN,tolS = getInputs()
    # 利用tolN,tolS，调用reDraw生成漂亮的图
    reDraw(tolS,tolN)


#组件Entry允许单行文本输入框；Check_button是复选框；IntVar是按钮整数值
root = Tk()
#Label(root,text="画图").grid(row=0,columnspan=3)#grid是确定组件的坐标
reDraw.f = Figure(figsize=(5,4), dpi=100) #create canvas
reDraw.canvas = FigureCanvasTkAgg(reDraw.f, master=root)
reDraw.canvas.show()
reDraw.canvas.get_tk_widget().grid(row=0, columnspan=3)


Label(root,text="tolN").grid(row=2,column=0)
tolNentry = Entry(root)
tolNentry.grid(row=1,column=1)
tolNentry.insert(0,'1.0')#插入默认值
# 点击“ReDraw”按钮后，调用drawNewTree()函数
Button(root,text="ReDraw",command=drawNewTree).grid(row=1,column=2,rowspan=3)
chkBtnVar = IntVar()
chkBtn = Checkbutton(root, text="Model Tree", variable = chkBtnVar)
chkBtn.grid(row=3, column=0, columnspan=2)#Checkbutton的状态需要创建一个变量，也就是IntVar

#读取数据,并初始化和reDraw()相关的全局变量
reDraw.rawDat = mat(loadDataSet('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch09/sine.txt'))
reDraw.testDat = arange(min(reDraw.rawDat[:,0]),max(reDraw.rawDat[:,0]),0.01)
reDraw(1.0, 10)

#Button(root,text="退出",fg="black",command=root.quit).grid(row=1,column=2)退出键可能不太好用
root.mainloop()



In [ ]:
testMat = mat(eye(4))
testMat

In [ ]:
mat0,mat1 = binSplitDataSet(testMat,1,0.5)

In [ ]:
mat0

In [ ]:
mat1

In [ ]:
dataMat= loadDataSet('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch09/ex00.txt')

dataMat

In [ ]:
createTree(dataMat)

In [ ]:
myDat1 = loadDataSet('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch09/ex0.txt')
myDat1

In [ ]:
createTree(myDat1)

In [ ]:
myDat2 = loadDataSet('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch09/ex2.txt')
myDat2

In [ ]:
createTree(myDat2)

In [ ]:
createTree(myDat2,ops=(10000,4))

In [ ]:
myTree = createTree(myDat2,ops=(0,1))
myDatTest = loadDataSet('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch09/ex2test.txt')
myMat2Test = mat(myDatTest)
prune(myTree,myMat2Test)

In [ ]:
myDat2 = mat(loadDataSet('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch09/exp2.txt'))
createTree(myDat2,modelLeaf,modelErr,(1,10))


In [ ]:
trainMat = mat(loadDataSet('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch09/bikeSpeedVsIq_train.txt'))
testMat = mat(loadDataSet('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch09/bikeSpeedVsIq_test.txt'))
myTree = createTree(trainMat,ops=(1,20))
yHat = createForeCast(myTree,testMat[:,0])

corrcoef(yHat,testMat[:,1],rowvar=0)[0,1]

In [ ]:
myTree1 = createTree(trainMat,modelLeaf,modelErr,(1,20))
yHat1 = createForeCast(myTree1,testMat[:,0],modelTreeEval)
corrcoef(yHat1,testMat[:,1],rowvar=0)[0,1]

In [ ]:
ws,X,Y = linearSolve(trainMat)
m,n = shape(testMat)
yHat2 = zeros((1,m))
for i in range(shape(testMat)[0]):
    yHat2[0][i] = testMat[i,0]*ws[1,0]+ws[0,0]
    
corrcoef(yHat2,testMat[:,1],rowvar=0)[0,1]


In [ ]:
import matplotlib.pyplot as plt
fig=plt.figure()
ax=fig.add_subplot(111)
#ax.scatter(array(trainMat)[:,0],array(trainMat)[:,1],c='cyan',marker='o')
ax.scatter(array(testMat)[:,0],array(testMat)[:,1],c='cyan',marker='o')
plt.show()